# Strategy Development with RustyBT

Learn how to build custom trading strategies from simple to complex.

**Strategies Covered:**
- Moving Average Crossover
- Mean Reversion
- Momentum Strategy
- Multi-Factor Strategy

**Estimated runtime:** 5 minutes

In [1]:
from rustybt.analytics import setup_notebook

setup_notebook()

import numpy as np

from rustybt import TradingAlgorithm
from rustybt.api import order_target_percent, symbol

✅ Notebook environment configured successfully
   - Async/await support enabled
   - Pandas display options optimized
   - Progress bars configured


## 1. Moving Average Crossover

In [2]:
class MovingAverageCrossover(TradingAlgorithm):
    def initialize(self, context) -> None:
        context.asset = symbol("SPY")
        context.fast_period = 20
        context.slow_period = 50
        context.prices = []

    def handle_data(self, context, data) -> None:
        price = data.current(context.asset, "close")
        context.prices.append(price)

        if len(context.prices) < context.slow_period:
            return

        fast_ma = np.mean(context.prices[-context.fast_period :])
        slow_ma = np.mean(context.prices[-context.slow_period :])

        current_position = context.portfolio.positions[context.asset].amount

        if fast_ma > slow_ma and current_position == 0:
            order_target_percent(context.asset, 1.0)
        elif fast_ma < slow_ma and current_position > 0:
            order_target_percent(context.asset, 0.0)

## 2. Mean Reversion Strategy

In [3]:
class MeanReversion(TradingAlgorithm):
    def initialize(self, context) -> None:
        context.asset = symbol("SPY")
        context.lookback = 20
        context.z_threshold = 2.0
        context.prices = []

    def handle_data(self, context, data) -> None:
        price = data.current(context.asset, "close")
        context.prices.append(price)

        if len(context.prices) < context.lookback:
            return

        mean = np.mean(context.prices[-context.lookback :])
        std = np.std(context.prices[-context.lookback :])
        z_score = (price - mean) / std if std > 0 else 0

        if z_score < -context.z_threshold:
            order_target_percent(context.asset, 1.0)
        elif z_score > context.z_threshold:
            order_target_percent(context.asset, 0.0)

## Next Steps

- Test strategies with real data (02_data_ingestion.ipynb)
- Optimize parameters (05_optimization.ipynb)
- Analyze performance (04_performance_analysis.ipynb)